In [56]:
import numpy as np
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

app = Dash(__name__)

def compute_fractal(xmin, xmax, ymin, ymax, func='mandelbrot', c=None, width=600, height=600, max_iter=200):
    x = np.linspace(xmin, xmax, width)
    y = np.linspace(ymin, ymax, height)
    X, Y = np.meshgrid(x, y)
    Z = X + 1j * Y
    C = Z.copy() if func == 'mandelbrot' else np.full_like(Z, c)

    escape = np.zeros(Z.shape, dtype=int)
    mask = np.ones(Z.shape, dtype=bool)

    for i in range(max_iter):
        Z[mask] = Z[mask]**2 + C[mask]
        escaped = np.abs(Z) > 2
        escape[mask & escaped] = i
        mask &= ~escaped

    return x, y, escape

def create_figure(x, y, fractal, bounds, title="Mandelbrot Fractal"):
    layout = dict(
        title=title,
        xaxis=dict(scaleanchor="y", scaleratio=1, constrain='domain'),
        yaxis=dict(autorange='reversed', constrain='domain'),
        margin=dict(l=10, r=10, t=40, b=10),
        width=600, height=600,
        dragmode='pan',
        updatemenus=[dict(buttons=[dict(label="Reset Zoom", method="relayout",
                                        args=[{"xaxis.range": bounds[:2], "yaxis.range": bounds[2:]}])])]
    )
    return go.Figure(data=go.Heatmap(z=fractal, x=x, y=y, colorscale='Inferno', showscale=False), layout=layout)

x, y, fractal = compute_fractal(*INITIAL_BOUNDS)
fig = create_figure(x, y, fractal, INITIAL_BOUNDS)

app.layout = html.Div([
    dcc.Graph(id='graph', figure=fig, config={'scrollZoom': True})
])


@app.callback(Output('graph', 'figure'), Input('graph', 'relayoutData'))
def update_fractal(relayoutData):
    if relayoutData and all(k in relayoutData for k in ['xaxis.range[0]', 'xaxis.range[1]', 'yaxis.range[0]', 'yaxis.range[1]']):
        bounds = [
            relayoutData['xaxis.range[0]'],
            relayoutData['xaxis.range[1]'],
            relayoutData['yaxis.range[0]'],
            relayoutData['yaxis.range[1]']
        ]
    else:
        bounds = INITIAL_BOUNDS

    x, y, fractal = compute_fractal(*bounds)
    return create_figure(x, y, fractal, INITIAL_BOUNDS)

if __name__ == '__main__':
    app.run(debug=True, port=8050)



In [57]:
xmin, xmax = -0.8, -0.7
ymin, ymax = 0.05, 0.15

x, y, fractal = mandelbrot_set(xmin, xmax, ymin, ymax)

fig = create_figure(x, y, fractal, INITIAL_BOUNDS)

fig.show(config={'scrollZoom': True})

In [58]:
c = -0.75 + 0.1j
bounds = [-1.5, 1.5, -1.5, 1.5]
x, y, fractal = compute_fractal(*bounds, func='julia', c=c)
fig = create_figure(x, y, fractal, bounds, title=f'Julia Fractal (c = {c})')
fig.show(config={'scrollZoom': True})